In [3]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
#from google.colab import files
from tqdm import *
#courses = pd.read_csv('courses_all.csv')
#career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])

In [2]:
eng_list = []

In [3]:
for i in tqdm(range(14)):
    page = rq.get('https://design.careers360.com/colleges/list-of-animation-fashion-colleges-in-India?page='+str(i))
    soup = bs(page.content,'html.parser')
    vals = soup.find_all('div',class_='title')
    for val in vals:
        try:
            eng_list.append(val.a['href'])
        except:
            pass

100%|██████████| 14/14 [01:16<00:00,  5.44s/it]


In [4]:
courses = pd.read_csv('design_courses.csv')

In [5]:
idx = 0
for link in tqdm(eng_list):
    try:
        page = rq.get(link)
        soup = bs(page.content,'html.parser')
        courses.loc[idx]=[link,soup.find_all('h4',class_='blockSubHeading')[0].get_text()[-2:],'https://design.careers360.com/'+soup.find_all('ul',class_='list-tabs')[0].find_all('li')[1].a['href']]
        idx+=1
    except:
        pass

100%|██████████| 419/419 [18:57<00:00,  2.72s/it]


In [10]:
courses = pd.read_csv('design_courses.csv')

















 55%|█████▌    | 11/20 [01:12<00:59,  6.59s/it]















 60%|██████    | 12/20 [01:12<00:48,  6.04s/it]















100%|██████████| 20/20 [01:12<00:00,  3.63s/it]




 45%|████▌     | 9/20 [01:12<01:29,  8.09s/it]




100%|██████████| 20/20 [01:12<00:00,  3.64s/it]










 45%|████▌     | 9/20 [01:13<01:29,  8.11s/it]










100%|██████████| 20/20 [01:13<00:00,  3.65s/it]


 40%|████      | 8/20 [01:13<01:49,  9.15s/it]


100%|██████████| 20/20 [01:13<00:00,  3.66s/it]





 70%|███████   | 14/20 [01:13<00:31,  5.22s/it]





100%|██████████| 20/20 [01:13<00:00,  3.66s/it]






 35%|███▌      | 7/20 [01:13<02:15, 10.46s/it]






100%|██████████| 20/20 [01:13<00:00,  3.66s/it]

In [10]:
from threading import Thread
def scrapper(start, end):
    career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])
    indx = 0
    cnt = start - 1
    for header_course in tqdm(courses['2'][start:end]):
        cnt += 1
        try:
            num = int(np.ceil(int(courses['1'][cnt])/10))
        except:
            continue
    #     print(f'num = {num}')
        for page_num in tqdm(range(num)):
            link = header_course + '?levelId=all&courseNid=all-all&page='+str(page_num)
    #         print(link)
            page = rq.get(link)
            course_soup = bs(page.content, 'html.parser')
        for course_link in course_soup.find_all('span',class_='readmore'):
            course_link = 'https://design.careers360.com/' + course_link.a['href']
            #             print(f'course_link = {course_link}')
            page1 = rq.get(course_link)
            soup = bs(page1.content,'html.parser')
            career360_courses.loc[indx]=None

            try:
                career360_courses.loc[indx]['Course'] = soup.find_all('h2',class_='block-title')[0].get_text()
                career360_courses.loc[indx]['College']=soup.find_all('h1',class_='titleNameCol')[0].get_text()
                career360_courses.loc[indx]['City'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[0].get_text()
                career360_courses.loc[indx]['State'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[1].get_text()
            except:
                pass

            try:
                career360_courses.loc[indx]['Eligiblity'] = soup.find_all('span',class_='more-eligibility')[0].get_text()[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Eligiblity'] = soup.find_all('div',class_='default-elig')[0].get_text()
                except:
                    career360_courses.loc[indx]['Eligiblity'] = None

            try:
                career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('span',class_='moreCourseDetails')[0].p.get_text())[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('div',class_='default-course-details')[0].get_text())
                except:
                    career360_courses.loc[indx]['Course_Detail'] = None

            for row in soup.find_all('div',class_='coursesPageLableInnerSec'):
                career360_courses.loc[indx][row.strong.get_text()] = row.p.get_text()
            indx+=1
    career360_courses.to_csv(f'career360_scrap_dm_law({start}-{end}).csv')

In [11]:
siz = courses.shape[0]
numer = int(siz/20)
prev = 0
def threader():
    threads = []
    for i in range(20):
        if i!=19:
            thread = Thread(target=scrapper, args = (i*numer,(i+1)*numer))
            threads.append(thread)
            thread.start()
        else:
            thread = Thread(target=scrapper, args = (19*numer,siz))
            threads.append(thread)
            thread.start()        
    
    for thread in threads:  # iterates over the threads
        thread.join()
                    
    print('done')

In [12]:
threader()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/20 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/20 [00:00<?, ?it/s]






  0%|          | 0/1 [00:00<?, ?it/s]







  0%|          | 0/20 [00:00<?, ?it/s]








  0%|          | 0/1 [00:00<?, ?it/s]









  0%|          | 0/20 [00:00<?, ?it/s]










  0%|          | 0/1 [00:00<?, ?it/s]











  0%|          | 0/20 [00:00<?, ?it/s]












  0%|          | 0/1 [00:00<?, ?it/s]













  0%|          | 0/20 [00:00<?, ?it/s]














  0%|          | 0/1 [00:00<?, ?it/s]















  0%|          | 0/20 [00:00<?, ?it/s]
















  0%|          | 0/1 [00:00<?, ?it/s]

















  0%|          | 0/20 [00:00<?, ?it/s]


















  0%|          | 0/1 [00:00<?, ?it/s]



















  0%|          | 0/20 [00:00<?, ?it/s]




















  0%|          | 0/1

 15%|█▌        | 3/20 [00:13<01:14,  4.41s/it]




























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.14s/it]







































100%|██████████| 1/1 [00:01<00:00,  1.55s/it]







































100%|██████████| 1/1 [00:00<00:00,  1.56it/s]







































100%|██████████| 1/1 [00:00<00:00,  2.01it/s]







































100%|██████████| 1/1 [00:00<00:00,  1.99it/s]







































100%|██████████| 1/1 [00:00<00:00,  1.93it/s]








































100%|██████████| 1/1 [00:00<00:00,  1.86it/s]









































100%|██████████| 1/1 [00:00<00:00,  1.80it/s]








































100%|██████████| 1/1 [00:01<00:00,  1.11s/it]







































100%|██████████| 1/1 [00:01<00:00,  1.21s/it]



















 15%|█▌        | 3/20 [00:15<01:29


 20%|██        | 4/20 [00:19<01:18,  4.88s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:01<00:00,  1.01s/it]



















 20%|██        | 4/20 [00:20<01:20,  5.02s/it][A




















  0%|          | 0/3 [00:00<?, ?it/s]





 10%|█         | 2/20 [00:20<03:00, 10.01s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]




















 33%|███▎      | 1/3 [00:00<00:00,  2.00it/s]



















100%|██████████| 1/1 [00:00<00:00,  1.21it/s]








































100%|██████████| 1/1 [00:01<00:00,  1.78s/it]








































 67%|██████▋   | 2/3 [00:02<00:01,  1.05s/it]










 20%|██        | 4/20 [00:22<01:28,  5.51s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]


 15%|█▌        | 3/20 [00:22<02:06,  7.43s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]

















 10%|█         | 2/20 [00:21<03:17, 10.99s/it]




 40%|████      | 4/10 [00:04<00:06,  1.03s/it]













 25%|██▌       | 5/20 [00:35<01:47,  7.15s/it]







 25%|██▌       | 5/20 [00:35<01:47,  7.18s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:01<00:00,  1.02s/it]







































 25%|██▌       | 5/20 [00:35<01:47,  7.15s/it]
























  0%|          | 0/7 [00:00<?, ?it/s]

























  0%|          | 0/1 [00:00<?, ?it/s]











 30%|███       | 6/20 [00:35<01:23,  5.99s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:00<00:00,  1.25it/s]









































100%|██████████| 1/1 [00:00<00:00,  1.62it/s]









































100%|██████████| 1/1 [00:00<00:00,  1.36it/s]











































100%|██████████| 1/1 [00:00<00:00,  1.11it/s]











































100%|█

  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:00<00:00,  1.00it/s]







































100%|██████████| 1/1 [00:00<00:00,  1.38it/s]







































 50%|█████     | 1/2 [00:00<00:00,  1.33it/s]











 40%|████      | 8/20 [00:47<01:11,  6.00s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:00<00:00,  2.32it/s]









































100%|██████████| 1/1 [00:00<00:00,  1.32it/s]








































100%|██████████| 2/2 [00:01<00:00,  1.01it/s]






































 35%|███▌      | 7/20 [00:49<01:31,  7.06s/it]



















 35%|███▌      | 7/20 [00:50<01:33,  7.23s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]













 40%|████      | 8/20 [00:50<01:15,  6.31s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00

...




















100%|██████████| 1/1 [00:00<00:00,  1.24it/s]
































 60%|██████    | 12/20 [01:50<01:13,  9.17s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:00<00:00,  2.11it/s]





































 90%|█████████ | 18/20 [01:50<00:12,  6.13s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]




















100%|██████████| 1/1 [00:00<00:00,  2.24it/s]







































100%|██████████| 1/1 [00:01<00:00,  1.76s/it]





































100%|██████████| 1/1 [00:01<00:00,  1.32s/it]


















 60%|██████    | 12/20 [01:52<01:14,  9.37s/it]A


















  0%|          | 0/1 [00:00<?, ?it/s]




 70%|███████   | 14/20 [01:53<00:48,  8.08s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]














 49%|████▊     | 18/37 [01:53<01:59,  6.29s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]


 

 10%|█         | 1/10 [00:00<00:05,  1.54it/s]











 85%|████████▌ | 17/20 [02:14<00:23,  7.93s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]
















100%|██████████| 1/1 [00:02<00:00,  2.32s/it]


































 20%|██        | 1/5 [00:01<00:06,  1.55s/it]














 30%|███       | 6/20 [02:15<05:15, 22.57s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]












 65%|██████▌   | 13/20 [02:15<01:12, 10.42s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]
















 20%|██        | 2/10 [00:02<00:08,  1.10s/it]



 80%|████████  | 16/20 [02:15<00:33,  8.48s/it]





















  0%|          | 0/2 [00:00<?, ?it/s]




 95%|█████████▌| 19/20 [02:15<00:07,  7.15s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]


















100%|██████████| 1/1 [00:00<00:00,  1.13it/s]



































 30%|███       | 3/10 [00:02<00:06,  1.01it/s]


















100%|█████
















100%|██████████| 1/1 [00:00<00:00,  1.95it/s]




















 65%|██████▌   | 13/20 [02:39<01:25, 12.26s/it]














  0%|          | 0/2 [00:00<?, ?it/s]

 90%|█████████ | 18/20 [02:39<00:17,  8.88s/it]















  0%|          | 0/1 [00:00<?, ?it/s]












 35%|███▌      | 7/20 [02:39<04:56, 22.83s/it]
















  0%|          | 0/1 [00:00<?, ?it/s]


 80%|████████  | 16/20 [02:40<00:40, 10.02s/it]

















  0%|          | 0/1 [00:00<?, ?it/s]














 50%|█████     | 1/2 [00:00<00:00,  1.58it/s]















100%|██████████| 1/1 [00:00<00:00,  2.11it/s]
































100%|██████████| 1/1 [00:00<00:00,  1.88it/s]































100%|██████████| 2/2 [00:01<00:00,  1.79it/s]





























100%|██████████| 1/1 [00:00<00:00,  1.49it/s]




























 95%|█████████▌| 19/20 [02:40<00:08,  8.47s/it]














  0%|          | 0/1 [00:00<?, ?it/s]














100%|█

 50%|█████     | 10/20 [03:30<03:30, 21.04s/it]







  0%|          | 0/1 [00:00<?, ?it/s]





 75%|███████▌  | 3/4 [00:01<00:00,  2.42it/s]






100%|██████████| 1/1 [00:00<00:00,  2.07it/s]









 75%|███████▌  | 15/20 [03:31<01:10, 14.07s/it]







  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00,  2.26it/s]












100%|██████████| 4/4 [00:01<00:00,  2.27it/s]









 92%|█████████▏| 34/37 [03:31<00:18,  6.21s/it]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00,  2.66it/s]







 90%|█████████ | 18/20 [03:32<00:23, 11.82s/it]






  0%|          | 0/1 [00:00<?, ?it/s]






100%|██████████| 1/1 [00:00<00:00,  2.62it/s]












100%|██████████| 1/1 [00:02<00:00,  2.22s/it]









 95%|█████████▍| 35/37 [03:33<00:12,  6.09s/it]





  0%|          | 0/1 [00:00<?, ?it/s]





100%|██████████| 1/1 [00:00<00:00,  2.15it/s]





 90%|█████████ | 18/20 [03:35<00:23, 11.97s/it]





  0%|          | 0/1 

done


In [14]:
scrapper(200,220)



  0%|          | 0/20 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:01<00:00,  1.13s/it]




  5%|▌         | 1/20 [00:02<00:46,  2.47s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  2.50it/s]




 10%|█         | 2/20 [00:06<01:00,  3.39s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  2.75it/s]




 15%|█▌        | 3/20 [00:09<00:52,  3.09s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  2.78it/s]




 20%|██        | 4/20 [00:10<00:40,  2.53s/it]


  0%|          | 0/2 [00:00<?, ?it/s]


 50%|█████     | 1/2 [00:01<00:01,  1.15s/it]


100%|██████████| 2/2 [00:01<00:00,  1.30it/s]




 25%|██▌       | 5/20 [00:15<00:45,  3.02s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:01<00:00,  1.09s/it]




 30%|███       | 6/20 [00:19<00:46,  3.31s/it]


  0%|          | 0/1 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00, 